In [ ]:
library(rjson)
library(ggplot2)
library(plyr)
library(stringr)
library(NLP)
library(tm)
library(tibble)
library(cluster)
library(factoextra)
library(NbClust)
library(RColorBrewer)
library(wordcloud)
library(topicmodels)

In [3]:
opinions <- read.csv("opinions_sample.csv", header=TRUE, sep=";")

In [12]:
dim(opinions)
#head(opinions)
names(opinions)

[1] 5000   17

[1] "absolute_url"        "author"              "author_str"         
 [4] "cluster"             "date_created"        "date_modified"      
 [7] "download_url"        "extracted_by_ocr"    "html"               
[10] "html_lawbox"         "html_with_citations" "joined_by"          
[13] "local_path"          "opinions_cited"      "per_curiam"         
[16] "plain_text"          "resource_uri"

In [15]:
opinions_text <- VCorpus(VectorSource(opinions$plain_text))
opinions_dtm <- DocumentTermMatrix(opinions_text)
inspect(opinions_dtm)

<<DocumentTermMatrix (documents: 5000, terms: 79148)>>
Non-/sparse entries: 290447/395449553
Sparsity           : 100%
Maximal term length: 73
Weighting          : term frequency (tf)
Sample             :
      Terms
Docs   and court for not see that  the this was with
  1088 495   428 240 202 107  606 1961  135  98  115
  1281 387   316 178 217 104  600 1476  102 203  117
  1687 334    78 154 166 110  443 1407  116  36   64
  195  574   201 317 238 159  593 1782  146  71  129
  241  581   149 325 198 126  371 2004   87  40  103
  2538 306   175  93 158  92  351 1294   84  47   74
  3094 460    69 151 145 108  384 1686  134  64  110
  4174 398   182 216 127 101  389 1045   88  70   82
  4187 594   381 208 201  57  540 2098  176 104  131
  4548 428    82 273 277 152  696 2378  169 236   87


In [16]:
#Remove stopwords from the corpus
opinions_text <- tm_map(opinions_text, removeWords, stopwords("english"))
opinions_dtm <- DocumentTermMatrix(opinions_text)
inspect(opinions_dtm)

<<DocumentTermMatrix (documents: 5000, terms: 77662)>>
Non-/sparse entries: 276505/388033495
Sparsity           : 100%
Maximal term length: 73
Weighting          : term frequency (tf)
Sample             :
      Terms
Docs   court federal id., j., opinion see state states the united
  1088   428      87   64  59      51 107   218     28 195     21
  1281   316     128  119  58      46 104   241     13 160     21
  1687    78      27   49  41      36 110   115     19 153     27
  195    201      27   45  68      66 159    16     38 164     39
  241    149      71   38  95      52 126     9     53 212     97
  2538   175      43   56  70      34  92    10     28 133     56
  3094    69       8   22 147      29 108    92     38 166     41
  4174   182       7   46  77      38 101    25     58 128     13
  4187   381      58   81  45      70  57   199     23 320     16
  4548    82      23   67  93      91 152    60     13 194     11


In [ ]:
#Remove Whitespace Punctuation and Numbers from the corpus
opinions_text <- tm_map(opinions_text, stripWhitespace)
opinions_text <- tm_map(opinions_text, removePunctuation)
opinions_text <- tm_map(opinions_text, removeNumbers)
opinions_dtm <- DocumentTermMatrix(opinions_text)
inspect(opinions_dtm)

In [ ]:
#TF-IDF representation of the corpus
opinions_dtm <- DocumentTermMatrix(opinions_text,
           control = list(weighting = function(x) weightTfIdf(x, normalize = FALSE),